***GENERATED CODE FOR shiipingcostregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderDate', 'transformation_label': 'String Indexer'}], 'feature': 'OrderDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '10', 'mean': '', 'stddev': '', 'min': '2020-07-13 05:25', 'max': '2020-12-12 15:16', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OrderDate'}, {'feature_label': 'OrderDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OrderDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Carrier', 'transformation_label': 'String Indexer'}], 'feature': 'Carrier', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': 'shopify', 'max': 'usps', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Carrier'}, {'feature_label': 'Carrier', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Carrier')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'LineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': '13FE0629-221A-4943-BC43-E45E8BA80189', 'max': 'DABCE544-7355-4E94-B759-6C0DD90C692F', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LineItemId'}, {'feature_label': 'LineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderId', 'transformation_label': 'String Indexer'}], 'feature': 'OrderId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': '11A43967-E75D-4EF2-BCC5-FB0DA339CC8A', 'max': 'E8A1B37E-CCF6-437F-9646-0CB0929989DE', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OrderId'}, {'feature_label': 'OrderId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OrderId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProductName', 'transformation_label': 'String Indexer'}], 'feature': 'ProductName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': 'Black Diamond 10 MM Dynex Runners - 120', 'max': 'THERM-A-REST® RIDGEREST® SOLITE™ - Regular / silver', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProductName'}, {'feature_label': 'ProductName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProductName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SKU', 'transformation_label': 'String Indexer'}], 'feature': 'SKU', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '5005033.5', 'stddev': '7065148.62', 'min': '10000848', 'max': 'R45AY 120', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SKU'}, {'feature_label': 'SKU', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SKU')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ref_OrderLineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'Ref_OrderLineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': 'LineItemRef_5592767430817', 'max': 'LineItemRef_6300144107681', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Ref_OrderLineItemId'}, {'feature_label': 'Ref_OrderLineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Ref_OrderLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ToZip', 'transformation_label': 'String Indexer'}], 'feature': 'ToZip', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '55605.89', 'stddev': '38937.92', 'min': '1760', 'max': '98368', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ToZip'}, {'feature_label': 'ToZip', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ToZip')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProductId', 'transformation_label': 'String Indexer'}], 'feature': 'ProductId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': '20B3D76826274DF587F6EBD66F5A7C3D', 'max': 'EEE58E10833C4305933C2F80EBF00D02', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProductId'}, {'feature_label': 'ProductId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProductId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FulfillmentLineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'FulfillmentLineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': '01A3FA83-5409-408D-91CE-24B3273C24FE', 'max': 'FE7F287B-1FCC-4248-975D-158710CCF347', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FulfillmentLineItemId'}, {'feature_label': 'FulfillmentLineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FulfillmentLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OrderLineItemId', 'transformation_label': 'String Indexer'}], 'feature': 'OrderLineItemId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': '13FE0629-221A-4943-BC43-E45E8BA80189', 'max': 'DABCE544-7355-4E94-B759-6C0DD90C692F', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'OrderLineItemId'}, {'feature_label': 'OrderLineItemId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OrderLineItemId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FulfillmentId', 'transformation_label': 'String Indexer'}], 'feature': 'FulfillmentId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': '20E16CC7-2709-4DF4-B410-68BC36144B64', 'max': 'F0BA34D2-7256-4796-B9A9-279CA0AACFAB', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FulfillmentId'}, {'feature_label': 'FulfillmentId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FulfillmentId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CarrierPickedUpDateTime', 'transformation_label': 'String Indexer'}], 'feature': 'CarrierPickedUpDateTime', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': '2020-07-13 05:25', 'max': '2021-02-09 07:21', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CarrierPickedUpDateTime'}, {'feature_label': 'CarrierPickedUpDateTime', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CarrierPickedUpDateTime')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CarrierDeliveredDateTime', 'transformation_label': 'String Indexer'}], 'feature': 'CarrierDeliveredDateTime', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '10', 'mean': '', 'stddev': '', 'min': '2020-07-16 04:14', 'max': '2021-03-09 07:21', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CarrierDeliveredDateTime'}, {'feature_label': 'CarrierDeliveredDateTime', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'CarrierDeliveredDateTime')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run shiipingcostregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	shippingcotdata = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1631533272453_ShippingCostData_130921.csv', 'filename': '1631533272453_ShippingCostData_130921.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/BiglynxUpdated/ShippingPriceData/ShippingCostData_130921.csv', 'viewFileName': 'ShippingCostData_130921.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run shiipingcostregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(shippingcotdata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "OrderDate", "transformation_label": "String Indexer"}], "feature": "OrderDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "2020-07-13 05:25", "max": "2020-12-12 15:16", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ShippingId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10", "mean": "2.2", "stddev": "0.92", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": "ShippingId"}, {"transformationsData": [{"feature_label": "Carrier", "transformation_label": "String Indexer"}], "feature": "Carrier", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "shopify", "max": "usps", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Carrier"}, {"transformationsData": [{"feature_label": "LineItemId", "transformation_label": "String Indexer"}], "feature": "LineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "13FE0629-221A-4943-BC43-E45E8BA80189", "max": "DABCE544-7355-4E94-B759-6C0DD90C692F", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LineItemId"}, {"transformationsData": [{"feature_label": "OrderId", "transformation_label": "String Indexer"}], "feature": "OrderId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "11A43967-E75D-4EF2-BCC5-FB0DA339CC8A", "max": "E8A1B37E-CCF6-437F-9646-0CB0929989DE", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderId"}, {"transformationsData": [{"feature_label": "ProductName", "transformation_label": "String Indexer"}], "feature": "ProductName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "Black Diamond 10 MM Dynex Runners - 120", "max": "THERM-A-REST\u00ae RIDGEREST\u00ae SOLITE\u2122 - Regular / silver", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProductName"}, {"transformationsData": [{"feature_label": "SKU", "transformation_label": "String Indexer"}], "feature": "SKU", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "5005033.5", "stddev": "7065148.62", "min": "10000848", "max": "R45AY 120", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SKU"}, {"transformationsData": [{"feature_label": "Ref_OrderLineItemId", "transformation_label": "String Indexer"}], "feature": "Ref_OrderLineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "LineItemRef_5592767430817", "max": "LineItemRef_6300144107681", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ref_OrderLineItemId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Qty", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10", "mean": "2.0", "stddev": "2.31", "min": "1", "max": "8", "missing": "0"}, "updatedLabel": "Qty"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Length", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10", "mean": "4.1", "stddev": "2.73", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "Length"}, {"transformationsData": [{"feature_label": "ToZip", "transformation_label": "String Indexer"}], "feature": "ToZip", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "55605.89", "stddev": "38937.92", "min": "1760", "max": "98368", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ToZip"}, {"transformationsData": [{"feature_label": "ProductId", "transformation_label": "String Indexer"}], "feature": "ProductId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "20B3D76826274DF587F6EBD66F5A7C3D", "max": "EEE58E10833C4305933C2F80EBF00D02", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProductId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FromZip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10", "mean": "98052.0", "stddev": "0.0", "min": "98052", "max": "98052", "missing": "0"}, "updatedLabel": "FromZip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RetailerId", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10", "mean": "375544787.0", "stddev": "0.0", "min": "375544787", "max": "375544787", "missing": "0"}, "updatedLabel": "RetailerId"}, {"transformationsData": [{"feature_label": "FulfillmentLineItemId", "transformation_label": "String Indexer"}], "feature": "FulfillmentLineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "01A3FA83-5409-408D-91CE-24B3273C24FE", "max": "FE7F287B-1FCC-4248-975D-158710CCF347", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FulfillmentLineItemId"}, {"transformationsData": [{"feature_label": "OrderLineItemId", "transformation_label": "String Indexer"}], "feature": "OrderLineItemId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "13FE0629-221A-4943-BC43-E45E8BA80189", "max": "DABCE544-7355-4E94-B759-6C0DD90C692F", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "OrderLineItemId"}, {"transformationsData": [{"feature_label": "FulfillmentId", "transformation_label": "String Indexer"}], "feature": "FulfillmentId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "20E16CC7-2709-4DF4-B410-68BC36144B64", "max": "F0BA34D2-7256-4796-B9A9-279CA0AACFAB", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FulfillmentId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "quantity", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10", "mean": "2.0", "stddev": "2.31", "min": "1", "max": "8", "missing": "0"}, "updatedLabel": "quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Weight", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "10", "mean": "2.57", "stddev": "4.44", "min": "0.12", "max": "14.39", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Weight"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Volume", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "10", "mean": "762.9", "stddev": "1339.58", "min": "20", "max": "4500", "missing": "0"}, "updatedLabel": "Volume"}, {"transformationsData": [{"feature_label": "CarrierPickedUpDateTime", "transformation_label": "String Indexer"}], "feature": "CarrierPickedUpDateTime", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "2020-07-13 05:25", "max": "2021-02-09 07:21", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CarrierPickedUpDateTime"}, {"transformationsData": [{"feature_label": "CarrierDeliveredDateTime", "transformation_label": "String Indexer"}], "feature": "CarrierDeliveredDateTime", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "10", "mean": "", "stddev": "", "min": "2020-07-16 04:14", "max": "2021-03-09 07:21", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CarrierDeliveredDateTime"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ShipCost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "10", "mean": "4.49", "stddev": "2.27", "min": "1.0", "max": "9.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ShipCost"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run shiipingcostregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["OrderDate_stringindexer", "ShippingId", "Carrier_stringindexer", "LineItemId_stringindexer", "OrderId_stringindexer", "ProductName_stringindexer", "SKU_stringindexer", "Ref_OrderLineItemId_stringindexer", "Qty", "Length", "ToZip_stringindexer", "ProductId_stringindexer", "FromZip", "RetailerId", "FulfillmentLineItemId_stringindexer", "OrderLineItemId_stringindexer", "FulfillmentId_stringindexer", "quantity", "Weight", "Volume", "CarrierPickedUpDateTime_stringindexer", "CarrierDeliveredDateTime_stringindexer"], "ShipCost")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

